In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
class RNN(d2l.Module):  # @save
    """The RNN model implemented with high-level APIs."""

    def __init__(self, num_inputs, num_hiddens):
        super().__init__()
        self.save_hyperparameters()
        self.rnn = nn.RNN(num_inputs, num_hiddens)

    def forward(self, inputs, H=None):
        return self.rnn(inputs, H)

In [3]:
class RNNLM(d2l.RNNLMScratch):  # @save
    """The RNN-based language model implemented with high-level APIs."""

    def init_params(self):
        self.linear = nn.LazyLinear(self.vocab_size)

    def output_layer(self, hiddens):
        return self.linear(hiddens).swapaxes(0, 1)

这段代码定义了一个名为`RNNLM`的类，它继承了`d2l.RNNLMScratch`类，并使用高级API实现了基于RNN的语言模型。以下是逐步解释：

1. `init_params(self)`: 这个方法用于初始化模型的参数。在这个实现中，我们使用PyTorch的`nn.LazyLinear`层代替手动定义的权重矩阵`self.W_hq`和偏置向量`self.b_q`。`nn.LazyLinear`层是一个线性层，它会在第一次前向传播时根据输入的形状自动确定权重矩阵的形状。在这里，我们将词汇表的大小作为输出维度传递给`nn.LazyLinear`。

2. `output_layer(self, hiddens)`: 这个方法接收RNN的隐藏状态，并通过一个线性层将其转换为最终的输出。与`RNNLMScratch`类中的实现不同，这里我们直接使用`self.linear`（一个`nn.LazyLinear`层）来实现线性变换。我们将隐藏状态`hiddens`输入到`self.linear`中，得到输出张量。然后，我们使用`swapaxes`方法交换输出张量的第0轴和第1轴，使其形状与`RNNLMScratch`类中的实现一致。

总的来说，这个类实现了一个基于RNN的语言模型，使用高级API替换了手动定义的权重矩阵和偏置向量。这使得代码更简洁，易于维护。

In [4]:
data = d2l.TimeMachine(batch_size=1024, num_steps=32)
rnn = RNN(num_inputs=len(data.vocab), num_hiddens=32)
model = RNNLM(rnn, vocab_size=len(data.vocab), lr=1)
model.predict('it has', 20, data.vocab)

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


'it hashsshsshsshsshsshsshs'

In [7]:
trainer = d2l.Trainer(max_epochs=100, gradient_clip_val=1, num_gpus=1)
trainer.fit(model, data)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
model.predict('it has', 20, data.vocab, d2l.try_gpu())

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


1. 高级API在深度学习框架中提供了标准RNN的实现。这些库可以帮助你避免浪费时间重新实现标准模型。此外，框架实现通常经过高度优化，与从头开始的实现相比，计算性能会有显著提升。

   使用高级API使RNN模型过拟合是可能的。过拟合通常是由于模型过于复杂，以至于它能够记住训练数据中的噪声而不仅仅是信号。为了使模型过拟合，你可以尝试以下方法：
   - 增加模型的复杂性，例如增加RNN的层数或隐藏单元数量。
   - 减少训练数据量，这样模型更容易记住训练数据中的特定细节。
   - 增加训练迭代次数，使模型在训练数据上训练更长时间。
   - 减少或取消正则化（如果有的话），如权重衰减或dropout。

2. 使用RNN实现第9.1节的自回归模型：

   要使用RNN实现自回归模型，你可以参考本节中的`RNNLMScratch`或`RNNLM`类的实现。自回归模型的主要思想是使用过去的观测值来预测未来的观测值。在这里，我们可以将RNN用作自回归模型，其中输入是过去的观测值序列，输出是未来观测值的预测。具体来说，你可以按照以下步骤实现自回归模型：

   - 定义一个RNN模型，可以使用标准RNN、GRU或LSTM等不同类型的RNN。
   - 初始化模型的权重和偏置。
   - 定义前向传播过程，将输入序列传递给RNN，并得到输出序列。
   - 使用适当的损失函数（如均方误差）计算预测值与真实值之间的差异。
   - 使用梯度下降法优化模型的参数。
   - 训练模型，并在验证集上评估性能。

   在实现自回归模型时，你可能需要根据具体问题调整模型的架构、损失函数和优化算法。